In [1]:
import pandas as pd 
import numpy as np 
import sys
import asyncio
import datetime
import os
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from dotenv import load_dotenv

sys.path.append('../scripts')

from telegram_scraper import scrape_telegram_channels, download_images_from_channels
from load_json_to_postgres import load_json_data_main, load_image_predictions
from object_detection import run_object_detection


In [ ]:
# Load all secretes 

from config import API_HOST, API_PORT, POSTGRES_USER

print(f"Connecting to DB as user: {POSTGRES_USER}")


## Data Scraping and Collection (Extract & Load)
Extract relevant text, image, price, product name, and timestamp data from public Telegram channels and save it to your raw data zone (e.g., JSON or Parquet files).

### Scraping Telegram data using json format

In [ ]:
# Run the async scraper function from notebook
await scrape_telegram_channels()

### Scraping Telegram image data 
Image Scraping from Telegram Channels

In [3]:
await download_images_from_channels()

🔌 Connecting to Telegram...
✅ Connected to Telegram.
🖼️  Downloading images from: https://t.me/Chemed123
✅ Downloaded 67 images from CheMed → data/images\CheMed123
🖼️  Downloading images from: https://t.me/lobelia4cosmetics
✅ Downloaded 1000 images from Lobelia pharmacy and cosmetics → data/images\lobelia4cosmetics
🖼️  Downloading images from: https://t.me/tikvahpharma
✅ Downloaded 299 images from Tikvah | Pharma → data/images\tikvahpharma


In [5]:
await scrape_telegram_channels()

🔌 Connecting to Telegram...
✅ Connected to Telegram.
🔍 Scraping messages from: https://t.me/Chemed123
✅ Saved 72 messages → data\raw\telegram_messages\2025-07-11\CheMed123.json
🔍 Scraping messages from: https://t.me/lobelia4cosmetics
✅ Saved 1000 messages → data\raw\telegram_messages\2025-07-11\lobelia4cosmetics.json
🔍 Scraping messages from: https://t.me/tikvahpharma
✅ Saved 999 messages → data\raw\telegram_messages\2025-07-11\tikvahpharma.json


### Organize your raw data in a partitioned directory structure
✅ Structured raw data lake (partitioned JSON)

✅ Organized image saving by channel

✅ Robust logging to logs/telegram_scraper.log and console

✅ Error handling and clear logging messages

In [3]:
await scrape_telegram_channels()

2025-07-11 16:36:21,567 | INFO | Scraping messages from: https://t.me/Chemed123
2025-07-11 16:36:22,309 | INFO | Saved 72 messages to data\raw\telegram_messages\2025-07-11\CheMed123.json
2025-07-11 16:36:22,309 | INFO | Scraping messages from: https://t.me/lobelia4cosmetics
2025-07-11 16:36:26,921 | INFO | Saved 1000 messages to data\raw\telegram_messages\2025-07-11\lobelia4cosmetics.json
2025-07-11 16:36:26,922 | INFO | Scraping messages from: https://t.me/tikvahpharma
2025-07-11 16:36:33,415 | INFO | Saved 999 messages to data\raw\telegram_messages\2025-07-11\tikvahpharma.json


## Data Modeling and Transformation (Transform)

Raw data is messy and untrustworthy. DBT helps us build a reliable "data factory" to transform this raw material into a clean, trusted data product. It allows us to:

●	Transform chaos into clean, structured tables.

●	Model the data into a standard star schema, creating a single source of truth for analytics.

●	Test our data to prevent bad data from reaching our users, building trust.

●	Modularize our logic, making the system scalable and easy to maintain.


## Create the DB on postgreSQL using DBname telegram_medical_db

In [ ]:
conn = psycopg2.connect(
    dbname="postgres",  # connect to default DB
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT")
)

conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = conn.cursor()
cursor.execute("CREATE DATABASE telegram_medical_db;")
cursor.close()
conn.close()
print("✅ Created telegram_medical_db")


### Check the connection to PostgreSQL

In [4]:
load_dotenv()  # Adjust path if needed

conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    dbname=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD")
)

print("✅ Connected to PostgreSQL")
conn.close()


✅ Connected to PostgreSQL


In [2]:
# Load today’s data (or pass a date like "2025-07-11")
load_json_data_main("2025-07-11")

sLoading: data\raw\telegram_messages\2025-07-11\CheMed123.json
sLoading: data\raw\telegram_messages\2025-07-11\lobelia4cosmetics.json
sLoading: data\raw\telegram_messages\2025-07-11\tikvahpharma.json
✅ Done. Inserted 2071 rows into raw.telegram_messages.


DBT (Data Build Tool) is an open-source tool that enables data analysts and engineers to transform raw data into clean, reliable datasets using only SQL. It operates on top of a data warehouse (e.g., PostgreSQL, BigQuery, Snowflake, Redshift) and helps build modular, tested, and documented data transformation pipelines.

## Develop dbt Models in Layers

Create Staging Models in dbt

mkdir -p models/staging
touch models/staging/stg_telegram_messages.sql


### Define our Staging Model (Cleaning + Restructuring)

### Run and Test

In [ ]:
# Run the model
# dbt run --select stg_telegram_messages

# Test the model
# dbt test --select stg_telegram_messages

## Data Enrichment with Object Detection (YOLO)

In [2]:
# Run YOLO prediction from notebook:

run_object_detection()



image 1/1 c:\Users\Specter\Documents\Tenx_Academy\Week-7\telegram-medical-insights\notebooks\data\images\CheMed123\CheMed123_10_1752237586.372846.jpg: 640x640 (no detections), 146.1ms
Speed: 5.9ms preprocess, 146.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\Specter\Documents\Tenx_Academy\Week-7\telegram-medical-insights\notebooks\data\images\CheMed123\CheMed123_10_1752240319.406987.jpg: 640x640 (no detections), 123.1ms
Speed: 3.9ms preprocess, 123.1ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\Specter\Documents\Tenx_Academy\Week-7\telegram-medical-insights\notebooks\data\images\CheMed123\CheMed123_10_1752240635.357972.jpg: 640x640 (no detections), 120.0ms
Speed: 7.0ms preprocess, 120.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\Specter\Documents\Tenx_Academy\Week-7\telegram-medical-insights\notebooks\data\images\CheMed123\CheMed123_11_1752237585.771509.jpg:

### Load results to PostgreSQL:

### Create the raw.image_detections Table

In [2]:
# Load environment variables
load_dotenv()

# PostgreSQL connection details
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DB = os.getenv("PG_DB")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")


def create_image_detections_table():
    conn = psycopg2.connect(
        dbname=PG_DB,
        user=PG_USER,
        password=PG_PASSWORD,
        host=PG_HOST,
        port=PG_PORT
    )
    
    with conn.cursor() as cur:
        cur.execute("""
            CREATE SCHEMA IF NOT EXISTS raw;

            CREATE TABLE IF NOT EXISTS raw.image_detections (
                message_id TEXT,
                detected_object_class TEXT,
                confidence_score FLOAT,
                channel TEXT,
                image_path TEXT
            );
        """)
        conn.commit()
        print("✅ Table raw.image_detections created or already exists.")
    
    conn.close()

In [4]:
create_image_detections_table()

✅ Table raw.image_detections created or already exists.


In [5]:
load_image_predictions()

✅ Loaded 2572 detections into raw.image_detections


# Build an Analytical API Fast API